# Imports

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import networkit as nk
import tqdm
import numpy as np
import glob
import time
import json

from sklearn.metrics import ndcg_score
from sklearn.preprocessing import MinMaxScaler, normalize
from scipy.stats import kendalltau

# Load data

In [2]:
openflights_df = pd.read_csv("../data/openflights/inf-openflights.edges", sep=" ", on_bad_lines="skip", index_col=False, header=None, names=["source", "target"])[1:]

openflights_df["source"] = openflights_df["source"].astype(int)
openflights_df["target"] = openflights_df["target"].astype(int)

openflights_df.head()

/var/folders/vf/np0zpbrx3l91mcfwk4mt1t2r0000gn/T/ipykernel_80132/1139279991.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  openflights_df = pd.read_csv("../data/openflights/inf-openflights.edges", sep=" ", on_bad_lines="skip", index_col=False, header=None, names=["source", "target"])[1:]


,source,target
1,1,2
2,3,4
3,3,2
4,3,5
5,3,6


# Generate networkit graph

In [3]:
g = nk.Graph(directed=False)

for row in openflights_df[["source", "target"]].to_records(index=False).tolist():
    g.addEdge(row[0], row[1], addMissing=True)

g.removeSelfLoops()

print("Number of nodes: ", g.numberOfNodes())
print("Number of edges: ", g.numberOfEdges())  

Number of nodes:  2940
Number of edges:  30501


# Network metrics

In [5]:
results = {
            "dataset": "openflights",
            "directed": False,
            "nodes": g.numberOfNodes(),
            "edges": g.numberOfEdges()
        }

# Density
results["density"] = nk.graphtools.density(g)
print(f"Density: ", results["density"])

# Average clustering coefficient
results["average_clustering_coefficient"] = nk.globals.ClusteringCoefficient().avgLocal(g, 10**6) 
print(f"Average clustering coefficient: ", results["average_clustering_coefficient"])

# Diameter
diameter = nk.distance.Diameter(g, algo=nk.distance.DiameterAlgo.Exact, nSamples=10**5)
diameter.run()
results["diameter"] = diameter.getDiameter() 
print(f"Diameter: ", results["diameter"])

Density:  0.007059877370478644
Average clustering coefficient:  0.3969607049654641
Diameter:  (14, 0)


# Centrality measures

In [6]:
def get_degree_centrality(g):
    start_time = time.process_time()
    
    degree = nk.centrality.DegreeCentrality(g)
    degree.run()
    
    end_time = time.process_time()
    
    return degree, (end_time - start_time)

def get_closeness_centrality(g):
    start_time = time.process_time()
    
    closeness = nk.centrality.Closeness(g, True, nk.centrality.ClosenessVariant.Generalized)
    closeness.run()
    
    end_time = time.process_time()
    
    return closeness, (end_time - start_time)

def get_topk_closeness_centrality(g, first_heu=False, second_heu=False, k=5):
    start_time = time.process_time()

    topk_closeness = nk.centrality.TopCloseness(g, k=k, first_heu=first_heu, sec_heu=second_heu)
    topk_closeness.run()
    
    end_time = time.process_time()

    return topk_closeness, (end_time - start_time)

centrality = {}
results["time_elapsed"] = {}

# Degree centrality
centrality["degree"], results["time_elapsed"]["degree"] = get_degree_centrality(g)

# Closeness centrality
centrality["closeness"], results["time_elapsed"]["closeness"] = get_closeness_centrality(g)

# Topk closeness centrality
ks = [5, 10, 50, 100, int(results["nodes"]/2)]

centrality["topkcloseness_0"] = {}
centrality["topkcloseness_1"] = {}
results["time_elapsed"]["topkcloseness_0"] = {}
results["time_elapsed"]["topkcloseness_1"] = {}

for k in ks:
    centrality["topkcloseness_0"][k], results["time_elapsed"]["topkcloseness_0"][k] = get_topk_closeness_centrality(g, False, False, k)
    centrality["topkcloseness_1"][k], results["time_elapsed"]["topkcloseness_1"][k] = get_topk_closeness_centrality(g, False, True, k)

In [7]:
results

{'dataset': 'openflights',
 'directed': False,
 'nodes': 2940,
 'edges': 30501,
 'density': 0.007059877370478644,
 'average_clustering_coefficient': 0.3969607049654641,
 'diameter': (14, 0),
 'time_elapsed': {'degree': 0.0006499999999993733,
  'closeness': 0.515244,
  'topkcloseness_0': {5: 0.061941000000000024,
   10: 0.10529699999999931,
   50: 0.23409500000000083,
   100: 0.5510199999999994,
   1470: 1.5937540000000006},
  'topkcloseness_1': {5: 0.303585,
   10: 0.41495700000000024,
   50: 0.5066600000000001,
   100: 0.4299660000000003,
   1470: 0.5656200000000009}}}

# Experiments

## Preprocess

In [8]:
normalised_scores = {}
nodes = {}

# Degree centrality
scaler = MinMaxScaler()
normalised_scores["degree"] = scaler.fit_transform(np.array([row[1] for row in centrality["degree"].ranking()]).reshape(-1, 1)).flatten()
nodes["degree"] = [row[0] for row in centrality["degree"].ranking()]

# Closeness centrality
scaler = MinMaxScaler()
normalised_scores["closeness"] = scaler.fit_transform(np.array([row[1] for row in centrality["closeness"].ranking()]).reshape(-1, 1)).flatten()
nodes["closeness"] = [row[0] for row in centrality["closeness"].ranking()]

# Topk closeness centrality
normalised_scores["topkcloseness_0"], normalised_scores["topkcloseness_1"] = {}, {}
nodes["topkcloseness_0"], nodes["topkcloseness_1"] = {}, {}

for k in ks:
    scaler = MinMaxScaler()
    normalised_scores["topkcloseness_0"][k] = scaler.fit_transform(np.array(centrality["topkcloseness_0"][k].topkScoresList()).reshape(-1, 1)).flatten()
    nodes["topkcloseness_0"][k] = centrality["topkcloseness_0"][k].topkNodesList()

    scaler = MinMaxScaler()
    normalised_scores["topkcloseness_1"][k] = scaler.fit_transform(np.array(centrality["topkcloseness_1"][k].topkScoresList()).reshape(-1, 1)).flatten()
    nodes["topkcloseness_1"][k] = centrality["topkcloseness_1"][k].topkNodesList()

## NDCG degree to (top-k) closeness

In [9]:
ndcg_scores = {}

for k in ks:
    ndcg_scores[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            y_score = np.where(np.isin(nodes[centrality_measure][k], nodes["degree"][:k]), normalised_scores[centrality_measure][k], np.zeros(k))
            y_true = normalised_scores["degree"][:k]
            
        else:
            y_score = np.where(np.isin(nodes[centrality_measure][:k], nodes["degree"][:k]), normalised_scores[centrality_measure][:k], np.zeros(k))
            y_true = normalised_scores["degree"][:k]

        ndcg_scores[k][centrality_measure] = ndcg_score([y_true], [y_score])
    
results["ndcg_degree_to"] = ndcg_scores

pd.DataFrame(ndcg_scores).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.995858,0.995858,0.995858
10,0.999071,0.999397,0.999397
50,0.998571,0.998642,0.998642
100,0.998847,0.998864,0.998864
1470,0.999543,0.999543,0.999543


## NDCG (top-k) closeness to degree

In [10]:
ndcg_scores = {}

for k in ks:
    ndcg_scores[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            # y_score = [normalised_scores["degree"][index] if node in nodes[centrality_measure][:k] else 0 for index, node in enumerate(nodes["degree"][:k])]
            y_score = np.where(np.isin(nodes["degree"][:k], nodes[centrality_measure][k]), normalised_scores["degree"][:k], np.zeros(k))
            y_true = normalised_scores[centrality_measure][k]

        else:
            y_score = np.where(np.isin(nodes["degree"][:k], nodes[centrality_measure][:k]), normalised_scores["degree"][:k], np.zeros(k))
            y_true = normalised_scores[centrality_measure][:k]
            
        ndcg_scores[k][centrality_measure] = ndcg_score([y_true], [y_score])
    
results["ndcg_to_degree"] = ndcg_scores

pd.DataFrame(ndcg_scores).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.999952,0.997818,0.997818
10,0.998847,0.973468,0.973468
50,0.999530,0.993065,0.993065
100,0.999644,0.995770,0.995770
1470,0.999769,0.998227,0.998227


## Kendall tau degree to (top k) closeness centrality correlation

In [11]:
kendalltau_corr = {}

for k in ks:
    kendalltau_corr[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            y_score = np.where(np.isin(nodes[centrality_measure][k], nodes["degree"][:k]), normalised_scores[centrality_measure][k], np.zeros(k))
            y_true = normalised_scores["degree"][:k]
            
        else:
            y_score = np.where(np.isin(nodes[centrality_measure][:k], nodes["degree"][:k]), normalised_scores[centrality_measure][:k], np.zeros(k))
            y_true = normalised_scores["degree"][:k]

        corr, _ = kendalltau([y_true], [y_score])
        kendalltau_corr[k][centrality_measure] = 0 if np.isnan(corr) else corr
    
results["kendalltau_degree_to"] = kendalltau_corr

pd.DataFrame(kendalltau_corr).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.737865,0.737865,0.737865
10,0.739985,0.881917,0.881917
50,0.785998,0.801556,0.801556
100,0.866432,0.871885,0.871885
1470,0.873577,0.873577,0.873577


## Kendall tau (top k) closeness to degree centrality correlation

In [12]:
kendalltau_corr = {}

for k in ks:
    kendalltau_corr[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        
        if "topk" in centrality_measure:
            # y_score = [normalised_scores["degree"][index] if node in nodes[centrality_measure][:k] else 0 for index, node in enumerate(nodes["degree"][:k])]
            y_score = np.where(np.isin(nodes["degree"][:k], nodes[centrality_measure][k]), normalised_scores["degree"][:k], np.zeros(k))
            y_true = normalised_scores[centrality_measure][k]

        else:
            y_score = np.where(np.isin(nodes["degree"][:k], nodes[centrality_measure][:k]), normalised_scores["degree"][:k], np.zeros(k))
            y_true = normalised_scores[centrality_measure][:k]

        corr, _ = kendalltau([y_true], [y_score])
        kendalltau_corr[k][centrality_measure] = 0 if np.isnan(corr) else corr
    
results["kendalltau_to_degree"] = kendalltau_corr

pd.DataFrame(kendalltau_corr).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.948683,0.948683,0.948683
10,0.501280,0.501280,0.501280
50,0.767112,0.767112,0.767112
100,0.839838,0.839838,0.839838
1470,0.850131,0.850131,0.850131


# Store results

In [13]:
with open(f"../results/{results['dataset']}_{results['directed']}.json", 'w') as outfile:
    json.dump(results, outfile)